In [ ]:
import parse_tfl_json
import google

In [ ]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime as sjt
where tfl_message = 'ok' and
 sjt.icscode not in (select icscode from tt_h.all_stations where in_sprawl = true) and 
 sjt.icscode in (select icscode from tt_h.all_stations where lat < 53.39926695813 and lng > -2.54)
"""

df = pd.read_sql(sql, conn)

In [ ]:
from process_journeys import get_station_lat_lng_from_icscode, get_london_icscodes, add_cycle_and_total_time, remove_journeys_not_arriving_clondon
from google import get_cycle_info
import parse_tfl_json
import json

In [ ]:
for r in df.iterrows():
    index = r[0]
    row = r[1]
    
    lat, lng = get_station_lat_lng_from_icscode(row["icscode"])

    df.loc[index, "depart_lat"] = lat
    df.loc[index, "depart_lng"] = lng
    
    try:
        j1 = row["tfl_response"]
        j2 = json.loads(j1)
    except:
        print("parse problem")
        continue
    
    try:
        journeys = j2["journeys"]
    except:
        print("no journeys in json")
        continue
    
    # Fill in journey summary so it exists even for journeys that don't end in central london
    journey = journeys[0]
    legs = journey["legs"]
    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary
    
    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)
    
    # Keep only journeys with a national rail component
    journeys = [j for j in journeys if len(j["legs"]) > 0]

    # Keep only journeys that end in central London  
    try:
        journeys = remove_journeys_not_arriving_clondon(journeys)
    except:
        print("Problem with journey starting at {}".format(row["station_name"]))
        continue

    
    if len(journeys) == 0:
        continue

    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)
    
    
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict"]]
df2 = df[cols]


In [ ]:
df3 = df2.sort_values(["station_name", "tfl_dest", "total_journeytime"]).drop_duplicates(["station_name", "tfl_dest"])

In [ ]:
df3.to_sql("parsed_rail_journeys", engine, schema="tt_h", if_exists="replace", index=False)

In [ ]:
sql = """
ALTER TABLE tt_h.parsed_rail_journeys ADD COLUMN id SERIAL PRIMARY KEY;
"""
cursor.execute(sql)
conn.commit()

In [ ]:
sql = """
SELECT AddGeometryColumn ('tt_h', 'parsed_rail_journeys', 'geom', 27700, 'POINT', 2);
UPDATE tt_h.parsed_rail_journeys SET geom = ST_Transform(ST_SetSRID(ST_MakePoint(depart_lng, depart_lat), 4326), 27700);
CREATE INDEX idx_geom_parsed_rail_journeys ON tt_h.parsed_rail_journeys USING gist(geom);

""" 
cursor.execute(sql)
conn.commit()

In [ ]:
sql = """
create view tt_h.parsed_rail_journeys_one_row_per_station_view as
select 
pf.nlc,
pf.icscode,
pf.station_name,
pf.tfl_message,
pf.tfl_dest,
pf.depart_lat,
pf.depart_lng,
pf.id,
pf.geom,
pf.natrail_journey_summary as natrail_journey_summary_pf,
pf.natrail_train_changes as natrail_train_changes_pf,
pf.final_arrival as final_arrival_pf,
pf.natrail_journey_minutes as natrail_journey_minutes_pf,
pf.arrive_lat as arrive_lat_pf,
pf.arrive_lng as arrive_lng_pf,
pf.total_journeytime as total_journeytime_pf ,
cw.natrail_journey_summary as natrail_journey_summary_cw,
cw.natrail_train_changes as natrail_train_changes_cw,
cw.final_arrival as final_arrival_cw,
cw.natrail_journey_minutes as natrail_journey_minutes_cw,
cw.arrive_lat as arrive_lat_cw,
cw.arrive_lng as arrive_lng_cw,
cw.total_journeytime as total_journeytime_cw
from
(select * from tt_h.parsed_rail_journeys where tfl_dest = 'E145HP') as cw
left join
(select * from tt_h.parsed_rail_journeys where tfl_dest = 'SW1H9AJ') as pf
on cw.nlc = pf.nlc
"""
cursor.execute(sql)
conn.commit()